<a href="https://colab.research.google.com/github/HromovAndrey/II_mood_6_part_3/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_text_classification2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання

* Завантажте усі датафрейми
* Добавте до кожного стовпчик `labels` з відповідним йому класом(bussiness, sports, education, etc). назви класів беріть з назв файлів
* Об'єднайте всі датафрейми в один, див [pd.concat](https://pandas.pydata.org/docs/reference/api/pandas.concat.html)
* Створіть стовпчик `text` в якому зберігатиметься текст для подальшої роботи. Можети вибрати такі варіанти:
  * Взяти один зі стовпчиків `headline`, `description`, `content`
  * Об'єднати тексти зі стовпчиків `headline`, `description`, `content`
* Видаліть стовпчики `headline`, `description`, `content`, `url`
* Розділіть дані на тренувальні та тестові
* Застосуйте `TfidfVectorizer`
* Натренуйте модель
* Виведіть результати

**Додатково**
* У `MyTokenizer` добавте обробку тексту за допомогою `nltk`, застусуйте один з варіантів:
  * стематизація
  * лематизація(використайте `nltk.pos_tag` для визначення частини мови)
  * обидва варіанти
* Використайте `n-gram` для класифікації(можливо доведеться збільшити `max_features`)
* Натренуйте різні моделі:
  * [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)
  * [GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB)
  * [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) з параметром `kernel="linear"`
  * [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) з параметром `kernel="rbf"`
  * [MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier)
* Використайте `optuna` для підбору гіперпараметрів


# Дані про статті

In [17]:
import pandas as pd

df1 = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module6/data/business_data.csv")
df2 = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module6/data/education_data.csv")
df3 = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module6/data/entertainment_data.csv")
df4 = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module6/data/sports_data.csv")
df5 = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module6/data/technology_data.csv")

In [18]:
!pip install optuna


In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from sklearn.base import BaseEstimator, TransformerMixin
from optuna import create_study
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [20]:
df1['labels'] = 'business'
df2['labels'] = 'education'
df3['labels'] = 'entertainment'
df4['labels'] = 'sports'
df5['labels'] = 'technology'

In [21]:
df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

In [22]:
df.head()

,headlines,description,content,url,category,labels
0,Nirmala Sitharaman to equal Morarji Desai’s re...,With the presentation of the interim budget on...,"Sitharaman, the first full-time woman finance ...",https://indianexpress.com/article/business/bud...,business,business
1,"‘Will densify network, want to be at least no....","'In terms of market share, we aim to double it...",The merger of Tata group’s budget airlines Air...,https://indianexpress.com/article/business/avi...,business,business
2,Air India group to induct an aircraft every si...,Air India currently has 117 operational aircra...,The Air India group plans to induct one aircra...,https://indianexpress.com/article/business/avi...,business,business
3,Red Sea woes: Exporters seek increased credit ...,Rising attacks forced shippers to consider the...,Indian exporters have asked the central govern...,https://indianexpress.com/article/business/red...,business,business
4,Air India group to induct a plane every 6 days...,"Apart from fleet expansion, 2024 will also see...",The Air India group plans to induct one aircra...,https://indianexpress.com/article/business/avi...,business,business


In [23]:
df['text'] = df['headlines'].fillna('') + ' ' + df['description'].fillna('') + ' ' + df['content'].fillna('')

In [24]:
df.drop(['headlines', 'description', 'content', 'url'], axis=1, inplace=True)


In [25]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['labels'], test_size=0.2, random_state=42)

In [26]:
class MyTokenizer(BaseEstimator, TransformerMixin):
    def __init__(self, lemmatize=True):
        self.lemmatize = lemmatize
        self.lemmatizer = WordNetLemmatizer()
        self.stemmer = PorterStemmer()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.apply(self._tokenize)

    def _tokenize(self, text):
        tokens = word_tokenize(text)
        if self.lemmatize:
            tokens = [self.lemmatizer.lemmatize(token, pos=self._get_wordnet_pos(token)) for token in tokens]
        else:
            tokens = [self.stemmer.stem(token) for token in tokens]
        return ' '.join(tokens)

    def _get_wordnet_pos(self, word):
        tag = pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": 'a', "N": 'n', "V": 'v', "R": 'r'}
        return tag_dict.get(tag, 'n')

In [27]:
tfidf = TfidfVectorizer(tokenizer=MyTokenizer().fit_transform, ngram_range=(1, 2), max_features=5000)

In [28]:
models = {
    'DecisionTree': DecisionTreeClassifier(),
    'GaussianNB': GaussianNB(),
    'SVC_linear': SVC(kernel='linear'),
    'SVC_rbf': SVC(kernel='rbf'),
    'MLPClassifier': MLPClassifier(max_iter=300)
}

In [29]:
def objective(trial):
    model_name = trial.suggest_categorical('model', list(models.keys()))
    model = models[model_name]

    # Настройка гиперпараметров
    if model_name == 'DecisionTree':
        model.max_depth = trial.suggest_int('max_depth', 1, 20)
    elif model_name == 'GaussianNB':
        # Для Naive Bayes нет значимых гиперпараметров
        pass
    elif model_name.startswith('SVC'):
        model.C = trial.suggest_loguniform('C', 1e-3, 1e3)
    elif model_name == 'MLPClassifier':
        model.hidden_layer_sizes = trial.suggest_int('hidden_layer_sizes', 50, 200)
        model.alpha = trial.suggest_loguniform('alpha', 1e-5, 1e-1)

    # Пайплайн
    pipeline = Pipeline([
        ('tfidf', tfidf),
        ('classifier', model)
    ])

    # Обучение модели
    pipeline.fit(X_train, y_train)

    # Предсказание и оценка модели
    preds = pipeline.predict(X_test)
    score = classification_report(y_test, preds, output_dict=True)['accuracy']

    return score

study = create_study(direction='maximize')
study.optimize(objective, n_trials=10)

print("Best hyperparameters:", study.best_params)

[I 2024-09-01 13:21:31,520] A new study created in memory with name: no-name-84245b79-beab-42a4-b049-0a1d802e5347
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
[W 2024-09-01 13:21:31,535] Trial 0 failed with parameters: {'model': 'DecisionTree', 'max_depth': 6} because of the following error: AttributeError("'str' object has no attribute 'apply'").
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-29-390d23187951>", line 24, in objective
    pipeline.fit(X_train, y_train)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 423, in fit
    Xt = self.

AttributeError: 'str' object has no attribute 'apply'